# 11-02: 音频处理与格式转换音频流处理、格式转换和 Web Audio API。

In [ ]:
// ========== 1. 使用 FFmpeg 进行音频转换 ==========
// 安装: npm install fluent-ffmpeg
// 需要系统安装 FFmpeg
import ffmpeg from 'fluent-ffmpeg';
import { createReadStream, createWriteStream } from 'fs';
import { Readable } from 'stream';
// MP3 转 WAV
function convertMp3ToWav(inputPath: string, outputPath: string): Promise<void> {
  return new Promise((resolve, reject) => {
    ffmpeg(inputPath)
      .toFormat('wav')
      .audioCodec('pcm_s16le')
      .audioChannels(2)
      .audioFrequency(44100)
      .on('end', () => {
        console.log('Conversion finished');
        resolve();
      })
      .on('error', (err) => {
        reject(err);
      })
      .save(outputPath);
  });
}
// 流式转换
function convertStream(inputStream: Readable, format: string): Readable {
  return ffmpeg(inputStream)
    .toFormat(format)
    .pipe() as Readable;
}
// 压缩音频
function compressAudio(
  inputPath: string,
  outputPath: string,
  bitrate: string = '128k'
): Promise<void> {
  return new Promise((resolve, reject) => {
    ffmpeg(inputPath)
      .audioBitrate(bitrate)
      .audioCodec('libmp3lame')
      .on('end', resolve)
      .on('error', reject)
      .save(outputPath);
  });
}
// 提取音频信息
function getAudioInfo(inputPath: string): Promise<any> {
  return new Promise((resolve, reject) => {
    ffmpeg.ffprobe(inputPath, (err, metadata) => {
      if (err) return reject(err);

      const audioStream = metadata.streams.find((s: any) => s.codec_type === 'audio');
      resolve({
        duration: metadata.format.duration,
        bitrate: metadata.format.bit_rate,
        format: metadata.format.format_name,
        codec: audioStream?.codec_name,
        sampleRate: audioStream?.sample_rate,
        channels: audioStream?.channels
      });
    });
  });
}

In [ ]:
// ========== 2. Web Audio API (Node.js 使用 web-audio-engine) ==========
// 安装: npm install web-audio-engine
import { AudioContext } from 'web-audio-engine';
// 创建音频上下文
const audioContext = new AudioContext();
// 加载音频文件
async function loadAudioFile(filePath: string): Promise<AudioBuffer> {
  const fs = await import('fs/promises');
  const data = await fs.readFile(filePath);

  // 解码音频
  const arrayBuffer = data.buffer.slice(
    data.byteOffset,
    data.byteOffset + data.byteLength
  );

  return audioContext.decodeAudioData(arrayBuffer);
}
// 音频处理：改变播放速度（不改变音调）
function changePlaybackRate(audioBuffer: AudioBuffer, rate: number): AudioBuffer {
  const source = audioContext.createBufferSource();
  source.buffer = audioBuffer;
  source.playbackRate.value = rate;

  const destination = audioContext.createMediaStreamDestination();
  source.connect(destination);

  // 创建新的缓冲区
  const newLength = Math.floor(audioBuffer.length / rate);
  const newBuffer = audioContext.createBuffer(
    audioBuffer.numberOfChannels,
    newLength,
    audioBuffer.sampleRate
  );

  for (let channel = 0; channel < audioBuffer.numberOfChannels; channel++) {
    const inputData = audioBuffer.getChannelData(channel);
    const outputData = newBuffer.getChannelData(channel);

    for (let i = 0; i < newLength; i++) {
      const inputIndex = Math.floor(i * rate);
      outputData[i] = inputData[inputIndex] || 0;
    }
  }

  return newBuffer;
}
// 调整音量
function adjustVolume(audioBuffer: AudioBuffer, volume: number): AudioBuffer {
  const newBuffer = audioContext.createBuffer(
    audioBuffer.numberOfChannels,
    audioBuffer.length,
    audioBuffer.sampleRate
  );

  for (let channel = 0; channel < audioBuffer.numberOfChannels; channel++) {
    const inputData = audioBuffer.getChannelData(channel);
    const outputData = newBuffer.getChannelData(channel);

    for (let i = 0; i < audioBuffer.length; i++) {
      outputData[i] = inputData[i] * volume;
    }
  }

  return newBuffer;
}
// 淡入淡出
function applyFade(audioBuffer: AudioBuffer, fadeInDuration: number, fadeOutDuration: number): AudioBuffer {
  const newBuffer = audioContext.createBuffer(
    audioBuffer.numberOfChannels,
    audioBuffer.length,
    audioBuffer.sampleRate
  );

  const sampleRate = audioBuffer.sampleRate;
  const fadeInSamples = Math.floor(fadeInDuration * sampleRate);
  const fadeOutSamples = Math.floor(fadeOutDuration * sampleRate);

  for (let channel = 0; channel < audioBuffer.numberOfChannels; channel++) {
    const inputData = audioBuffer.getChannelData(channel);
    const outputData = newBuffer.getChannelData(channel);

    for (let i = 0; i < audioBuffer.length; i++) {
      let gain = 1.0;

      // 淡入
      if (i < fadeInSamples) {
        gain = i / fadeInSamples;
      }
      // 淡出
      else if (i > audioBuffer.length - fadeOutSamples) {
        gain = (audioBuffer.length - i) / fadeOutSamples;
      }

      outputData[i] = inputData[i] * gain;
    }
  }

  return newBuffer;
}

In [ ]:
// ========== 3. 音频流处理 ==========
import { Transform } from 'stream';
// 音频流转换器
class AudioTransform extends Transform {
  private buffer: Buffer = Buffer.alloc(0);
  private readonly sampleSize: number = 2; // 16-bit

  constructor(private options: {
    volume?: number;
    sampleRate?: number;
  }) {
    super();
  }

  _transform(chunk: Buffer, encoding: string, callback: Function): void {
    this.buffer = Buffer.concat([this.buffer, chunk]);

    // 处理完整的样本
    const samplesToProcess = Math.floor(this.buffer.length / this.sampleSize);
    const processedLength = samplesToProcess * this.sampleSize;

    if (samplesToProcess > 0) {
      const output = this.processSamples(
        this.buffer.slice(0, processedLength),
        samplesToProcess
      );

      this.push(output);
      this.buffer = this.buffer.slice(processedLength);
    }

    callback();
  }

  private processSamples(data: Buffer, sampleCount: number): Buffer {
    const output = Buffer.alloc(data.length);
    const volume = this.options.volume ?? 1.0;

    for (let i = 0; i < sampleCount; i++) {
      const offset = i * this.sampleSize;
      let sample = data.readInt16LE(offset);

      // 应用音量
      sample = Math.max(-32768, Math.min(32767, Math.floor(sample * volume)));

      output.writeInt16LE(sample, offset);
    }

    return output;
  }
}
// 使用流处理
function processAudioStream(
  inputPath: string,
  outputPath: string,
  options: { volume?: number }
): void {
  const input = createReadStream(inputPath);
  const transform = new AudioTransform(options);
  const output = createWriteStream(outputPath);

  input.pipe(transform).pipe(output);
}
// 流式音频播放器
class AudioPlayer {
  private currentStream?: Readable;

  play(stream: Readable): void {
    this.stop();
    this.currentStream = stream;

    // 实际播放需要音频输出设备
    // 这里只是示例
    stream.on('data', (chunk) => {
      // 将数据发送到音频输出
    });

    stream.on('end', () => {
      console.log('Playback finished');
    });
  }

  stop(): void {
    if (this.currentStream) {
      this.currentStream.destroy();
      this.currentStream = undefined;
    }
  }

  pause(): void {
    this.currentStream?.pause();
  }

  resume(): void {
    this.currentStream?.resume();
  }
}

In [ ]:
// ========== 4. 音频分割与合并 ==========
// 分割音频
function splitAudio(
  inputPath: string,
  segmentDuration: number,
  outputPattern: string
): Promise<string[]> {
  return new Promise((resolve, reject) => {
    const outputFiles: string[] = [];

    ffmpeg(inputPath)
      .outputOptions([
        `-f segment`,
        `-segment_time ${segmentDuration}`,
        `-c copy`
      ])
      .output(outputPattern)
      .on('end', () => resolve(outputFiles))
      .on('error', reject)
      .run();
  });
}
// 合并音频文件
function mergeAudioFiles(
  inputFiles: string[],
  outputPath: string
): Promise<void> {
  return new Promise((resolve, reject) => {
    const command = ffmpeg();

    inputFiles.forEach((file) => {
      command.input(file);
    });

    command
      .on('end', resolve)
      .on('error', reject)
      .mergeToFile(outputPath, '/tmp');
  });
}
// 提取音频片段
function extractSegment(
  inputPath: string,
  startTime: number,
  duration: number,
  outputPath: string
): Promise<void> {
  return new Promise((resolve, reject) => {
    ffmpeg(inputPath)
      .setStartTime(startTime)
      .setDuration(duration)
      .output(outputPath)
      .on('end', resolve)
      .on('error', reject)
      .run();
  });
}
// 拼接音频（带淡入淡出）
function concatenateWithTransitions(
  inputFiles: string[],
  outputPath: string,
  transitionDuration: number = 2
): Promise<void> {
  return new Promise((resolve, reject) => {
    const filterComplex = inputFiles
      .map((_, i) => `[${i}:a]`)
      .join('') +
      `concat=n=${inputFiles.length}:v=0:a=1[outa]`;

    const command = ffmpeg();

    inputFiles.forEach((file) => {
      command.input(file);
    });

    command
      .complexFilter(filterComplex, ['outa'])
      .audioCodec('libmp3lame')
      .audioBitrate('128k')
      .output(outputPath)
      .on('end', resolve)
      .on('error', reject)
      .run();
  });
}

In [ ]:
// ========== 5. 实时音频流 (WebSocket) ==========
import { WebSocket } from 'ws';
interface AudioStreamConfig {
  sampleRate: number;
  channels: number;
  bitDepth: number;
  bufferSize: number;
}
class RealtimeAudioStreamer {
  private ws?: WebSocket;
  private config: AudioStreamConfig = {
    sampleRate: 16000,
    channels: 1,
    bitDepth: 16,
    bufferSize: 4096
  };

  connect(url: string): void {
    this.ws = new WebSocket(url);

    this.ws.on('open', () => {
      console.log('Connected to audio server');
      // 发送配置
      this.ws?.send(JSON.stringify({ type: 'config', ...this.config }));
    });

    this.ws.on('message', (data) => {
      // 接收音频数据
      if (data instanceof Buffer) {
        this.onAudioData?.(data);
      } else {
        // 处理控制消息
        const message = JSON.parse(data.toString());
        this.onControlMessage?.(message);
      }
    });
  }

  sendAudioData(data: Buffer): void {
    if (this.ws?.readyState === WebSocket.OPEN) {
      this.ws.send(data);
    }
  }

  onAudioData?: (data: Buffer) => void;
  onControlMessage?: (message: any) => void;

  disconnect(): void {
    this.ws?.close();
  }
}
// PCM 格式音频编码器
class PCMEncoder {
  constructor(
    private sampleRate: number,
    private channels: number,
    private bitDepth: number
  ) {}

  encode(samples: Float32Array): Buffer {
    const bytesPerSample = this.bitDepth / 8;
    const buffer = Buffer.alloc(samples.length * bytesPerSample);

    for (let i = 0; i < samples.length; i++) {
      const sample = Math.max(-1, Math.min(1, samples[i]));

      if (this.bitDepth === 16) {
        buffer.writeInt16LE(Math.floor(sample * 32767), i * bytesPerSample);
      } else if (this.bitDepth === 32) {
        buffer.writeInt32LE(Math.floor(sample * 2147483647), i * bytesPerSample);
      }
    }

    return buffer;
  }

  decode(buffer: Buffer): Float32Array {
    const bytesPerSample = this.bitDepth / 8;
    const samples = new Float32Array(buffer.length / bytesPerSample);

    for (let i = 0; i < samples.length; i++) {
      if (this.bitDepth === 16) {
        samples[i] = buffer.readInt16LE(i * bytesPerSample) / 32767;
      } else if (this.bitDepth === 32) {
        samples[i] = buffer.readInt32LE(i * bytesPerSample) / 2147483647;
      }
    }

    return samples;
  }
}